In [2]:
import sys
import os
import cv2
import tensorflow.keras.backend as K
import numpy as np
from numpy import asarray
from core.networks_simple import UNET
from PIL import Image as im
K.set_image_data_format('channels_first')
os.environ['KERAS_BACKEND'] = "tensorflow"

def Synthesis(width, height,path1, path2, out_path):
  vid_arr = np.zeros(shape=(2, height, width, 3), dtype="uint8")
  # image1 = im.open(path1)
  image1 = cv2.cvtColor(cv2.imread(path1), cv2.COLOR_BGR2RGB)
  # image2 = im.open(path2)
  image2 = cv2.cvtColor(cv2.imread(path2), cv2.COLOR_BGR2RGB)
  vid_arr[0] = cv2.resize(asarray(image1), (width, height))
  vid_arr[1] = cv2.resize(asarray(image2), (width, height))



  model = UNET((6, height, width))
  model.load_weights('./model_weights_for 512x256 5ep simple unet/model_weights.hdf5')



    
  pred = model.predict(np.expand_dims(np.transpose(np.concatenate((vid_arr[0], vid_arr[1]), axis=2)/255., (2,0,1)), axis=0))
  res = (np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")
  img = im.fromarray(res)
  img.save(out_path)


In [3]:
import math
from skimage.metrics import structural_similarity as compare_ssim
def calculateMetrics(gt, result):
  psnr(gt, result)
  ssim = calculate_ssim(gt, result)
  print(f"SSIM: {ssim}%")

def psnr(img1, img2):
  mse = np.mean((img1 - img2) ** 2)
  if mse == 0:
    return 100
  # print(f"MSE: {mse}")
  PIXEL_MAX = 255.0
  psnr_val = 20 * math.log10(PIXEL_MAX / math.sqrt(mse))
  print(f"PSNR: {psnr_val}")

def calculate_ssim(gt, result):
  grayGT = cv2.cvtColor(gt, cv2.COLOR_BGR2GRAY)
  grayRes = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
  (score, diff) = compare_ssim(grayGT, grayRes, full=True)
  diff = (diff * 255).astype("uint8")
  return score

In [4]:
from PIL import Image
def adjuster(r1, r2, diff, gt, image):
  mask1 = np.where((diff >= r1) & (diff < (r1 + 1)))
  mask2 = np.where((diff >= r2) & (diff < (r2 + 1)))
  print(mask1[0].size, mask2[0].size)
  image[mask1] += r1
  image[mask2] -= r1
  diff_array = np.absolute(np.array(gt) - np.array(image))
  unique, counts = np.unique(diff_array, return_counts=True)
  print(unique)
  print(counts)
  print(f"For Val: {r1} and {r2 - 256}")
  calculateMetrics(gt, image)
  modified_img = Image.fromarray(image, 'RGB')
  return modified_img

#Balloon

In [4]:
Synthesis(928, 832, './Results/Balloon/Ref/000350.png', './Results/Balloon/Ref/000352.png', './Results/Balloon/NC/Balloon_NC.png')

In [5]:
BalloonGT = cv2.imread('./Results/Balloon/Ref/000351.png')
BalloonGT = cv2.resize(BalloonGT, (928, 832))
BalloonNC = cv2.imread(f'./Results/Balloon/NC/Balloon_NC.png')
calculateMetrics(BalloonGT, BalloonNC)

PSNR: 33.5819316174597
SSIM: 0.868229288696316%


In [7]:
GT = cv2.imread('./Results/Balloon/Ref/000351.png')
GT = cv2.resize(GT, (928, 832))
GT = cv2.cvtColor(GT, cv2.COLOR_BGR2RGB)
PRED = cv2.cvtColor(cv2.imread('./Results/Balloon/NC/Balloon_NC.png'), cv2.COLOR_BGR2RGB)
diff = np.absolute(np.array(GT) - np.array(PRED))
for i in reversed(range(180, 256)):
  if i == 180:
    result_img = adjuster(256 - i, i, diff, GT, PRED)
  adjuster(256 - i, i, diff, GT, PRED)
result_img.save('./Results/Balloon/CC/Balloon_CC.png')

214833 220679
[  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 

#Bear

In [5]:
Synthesis(1920, 1056, './Results/Bear/Ref/00000.jpg', './Results/Bear/Ref/00002.jpg', './Results/Bear/NC/BearNC.png')

In [7]:
BearGT = cv2.imread('./Results/Bear/Ref/00001.jpg')
BearGT = cv2.resize(BearGT, (1920, 1056))
BearNC = cv2.imread(f'./Results/Bear/NC/BearNC.png')
calculateMetrics(BearGT, BearNC)

PSNR: 29.74257665114379
SSIM: 0.4597769375084055%


In [8]:
GT = cv2.imread('./Results/Bear/Ref/00001.jpg')
GT = cv2.resize(GT, (1920, 1056))
GT = cv2.cvtColor(GT, cv2.COLOR_BGR2RGB)
PRED = cv2.cvtColor(cv2.imread('./Results/Bear/NC/BearNC.png'), cv2.COLOR_BGR2RGB)
diff = np.absolute(np.array(GT) - np.array(PRED))
for i in reversed(range(180, 256)):
  if i == 180:
    result_img = adjuster(256 - i, i, diff, GT, PRED)
  adjuster(256 - i, i, diff, GT, PRED)
result_img.save('./Results/Bear/CC/BearCC.png')

235636 222259
[  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 

#Boat

In [9]:
Synthesis(448, 256, './Results/Boat/Ref/im1.png', './Results/Boat/Ref/im3.png', './Results/Boat/NC/BoatNC.png')


In [10]:
BalloonGT = cv2.imread('./Results/Boat/Ref/im2.png')
BalloonGT = cv2.resize(BalloonGT, (448, 256))
BalloonNC = cv2.imread(f'./Results/Boat/NC/BoatNC.png')
calculateMetrics(BalloonGT, BalloonNC)


PSNR: 29.89849939288391
SSIM: 0.6507057363428828%


In [11]:
GT = cv2.imread('./Results/Boat/Ref/im2.png')
GT = cv2.resize(GT, (448, 256))
GT = cv2.cvtColor(GT, cv2.COLOR_BGR2RGB)
PRED = cv2.cvtColor(cv2.imread('./Results/Boat/NC/BoatNC.png'), cv2.COLOR_BGR2RGB)
diff = np.absolute(np.array(GT) - np.array(PRED))
for i in reversed(range(180, 256)):
  if i == 180:
    result_img = adjuster(256 - i, i, diff, GT, PRED)
  adjuster(256 - i, i, diff, GT, PRED)
result_img.save('./Results/Boat/CC/BoatCC.png')

12949 14618
[  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 24

#Swing

In [12]:
Synthesis(448, 256, './Results/Swing/Ref/im1.png', './Results/Swing/Ref/im3.png', './Results/Swing/NC/SwingNC.png')


In [13]:

BalloonGT = cv2.imread('./Results/Swing/Ref/im2.png')
BalloonGT = cv2.resize(BalloonGT, (448, 256))
BalloonNC = cv2.imread(f'./Results/Swing/NC/SwingNC.png')
calculateMetrics(BalloonGT, BalloonNC)


PSNR: 29.0936826261975
SSIM: 0.627814733430737%


In [14]:

GT = cv2.imread('./Results/Swing/Ref/im2.png')
GT = cv2.resize(GT, (448, 256))
GT = cv2.cvtColor(GT, cv2.COLOR_BGR2RGB)
PRED = cv2.cvtColor(cv2.imread('./Results/Swing/NC/SwingNC.png'), cv2.COLOR_BGR2RGB)
diff = np.absolute(np.array(GT) - np.array(PRED))
for i in reversed(range(180, 256)):
  if i == 180:
    result_img = adjuster(256 - i, i, diff, GT, PRED)
  adjuster(256 - i, i, diff, GT, PRED)
result_img.save('./Results/Swing/CC/SwingCC.png')


8536 9049
[  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 